In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [6]:
import pandas as pd

# Load datasets
data_fake1 = pd.read_csv("C:/Users/Moksha's Lappy/Desktop/GFG/Fake.csv")
data_true1 = pd.read_csv("C:/Users/Moksha's Lappy/Desktop/GFG/True.csv")

data_fake2 = pd.read_csv("C:/Users/Moksha's Lappy/Desktop/GFG/clean_fake_facts_5000.csv")
data_true2 = pd.read_csv("C:/Users/Moksha's Lappy/Desktop/GFG/clean_true_facts_5000 (2).csv")

data_fake2.columns = data_fake1.columns  
data_true2.columns = data_true1.columns  

common_columns = list(set(data_fake1.columns) & set(data_fake2.columns))
data_fake1 = data_fake1[common_columns]
data_fake2 = data_fake2[common_columns]

common_columns = list(set(data_true1.columns) & set(data_true2.columns))
data_true1 = data_true1[common_columns]
data_true2 = data_true2[common_columns]


# Merge fake datasets
merged_fake = pd.concat([data_fake1, data_fake2], ignore_index=True)
merged_fake = merged_fake.sample(frac=1).reset_index(drop=True)  # Shuffle data

# Merge true datasets
merged_true = pd.concat([data_true1, data_true2], ignore_index=True)
merged_true = merged_true.sample(frac=1).reset_index(drop=True)  # Shuffle data

# Save merged datasets
merged_fake.to_csv("C:/Users/Moksha's Lappy/Desktop/GFG/merged_fake.csv", index=False)
merged_true.to_csv("C:/Users/Moksha's Lappy/Desktop/GFG/merged_true.csv", index=False)


# Display the first 5 rows of each dataset
merged_fake.head()
merged_true.head()

,text,title,subject,date
0,The U.S. Constitution was signed in 1787.,The U.S. Constitution was signed in 1787....,Politics,25-08-2000
1,GAZA (Reuters) - The Palestinian Islamist grou...,Hamas picks new deputy chief whom Israel blame...,worldnews,"October 5, 2017"
2,NEW YORK (Reuters) - New York Mayor Bill de Bl...,"NYC mayor proposes retirement plan, light-rail...",politicsNews,"February 5, 2016"
3,VATICAN CITY/WASHINGTON (Reuters) - U.S. Democ...,Papal official denies report Sanders invited h...,politicsNews,"April 8, 2016"
4,"LAHORE, Pakistan (Reuters) - Pakistanis began ...",Pakistanis vote in by-election seen as test of...,worldnews,"September 17, 2017"


In [7]:
merged_fake["class"] = 0  # Label fake news as 0
merged_true["class"] = 1  # Label true news as 1

In [8]:
merged_fake.shape, merged_true.shape


((28302, 5), (26263, 5))

In [9]:
# Extract last 10 rows for manual testing
data_fake_manual_testing = merged_fake.tail(10)
data_true_manual_testing = merged_true.tail(10)

# Remove last 10 rows from the datasets
data_fake = merged_fake.iloc[:-10]
data_true = merged_true.iloc[:-10]


In [10]:
data_fake_manual_testing['class'] = 0
data_true_manual_testing['class'] = 1


C:\Users\Moksha's Lappy\AppData\Local\Temp\ipykernel_28720\4264077654.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_manual_testing['class'] = 0
C:\Users\Moksha's Lappy\AppData\Local\Temp\ipykernel_28720\4264077654.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_manual_testing['class'] = 1


In [11]:
data_merge = pd.concat([data_fake, data_true], axis=0)
data_merge.head(10)


,text,title,subject,date,class
0,One would think that conservatives would have ...,"‘March 4 Trump’ Gets HILARIOUSLY Bad Turnout,...",News,"March 4, 2017",0
1,In between defending white supremacists and pa...,The Guy Donald Trump Called ‘My African-Ameri...,News,"August 31, 2017",0
2,With Reince Priebus out as White House chief o...,Pathetic: Trump Actually Needs A General To B...,News,"August 5, 2017",0
3,It s hard for millennials to escape the leftis...,IS LAW AND ORDER SVU PANDERING To Cop Hating M...,left-news,"Oct 14, 2015",0
4,,CHILLING! FOX REPORTER JAMES ROSEN Recounts Be...,Government News,"Mar 6, 2017",0
5,It is no secret that President Donald Trump et...,"White House Staffer Admits, “This S***’s Hard...",News,"April 27, 2017",0
6,Elton John will perform at Donald Trump s inau...,KEY TRUMP ADVISOR: ELTON JOHN Will Perform At ...,politics,"Nov 23, 2016",0
7,Tune in to the Alternate Current Radio Network...,Boiler Room EP #117 – Straight Outta Tavistock...,US_News,"July 14, 2017",0
8,The FIFA World Cup is held every two years.,The FIFA World Cup is held every two years....,Current Affairs,07-09-2021,0
9,It s time for the media to start doing their j...,WHY IS THE MEDIA HIDING This Endorsement?…KKK ...,left-news,"Sep 10, 2016",0


In [12]:
data = data_merge.drop(['title', 'subject', 'date'], axis=1)


In [13]:
import re
import string

def wordopt(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)  # Use raw string (r"...") to avoid warnings
    text = re.sub(r"\W", " ", text)  # Replace non-word characters with spaces
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
    text = re.sub(r'<.*?>+', '', text)  # Remove HTML tags
    text = re.sub(r"[%s]" % re.escape(string.punctuation), '', text)  # Remove punctuation
    text = re.sub(r"\n", "", text)  # Remove newline characters
    text = re.sub(r"\w*\d\w*", '', text)  # Remove words containing numbers
    return text


In [14]:
# Assuming 'data' is your merged DataFrame
data['text'] = data['text'].fillna('')  # Handle missing values
data['text'] = data['text'].apply(wordopt)

# Splitting into features and labels
x = data['text']
y = data['class']

In [15]:
from sklearn.model_selection import train_test_split

# Splitting dataset into training (75%) and testing (25%) sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

# Displaying the sizes of the splits
print(f"Training set size: {len(x_train)}")
print(f"Testing set size: {len(x_test)}")


Training set size: 40908
Testing set size: 13637


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TfidfVectorizer
vectorization = TfidfVectorizer()

# Fit and transform training data
xv_train = vectorization.fit_transform(x_train)

# Transform testing data
xv_test = vectorization.transform(x_test)

# Display the shape of transformed data
print(f"Training data shape: {xv_train.shape}")
print(f"Testing data shape: {xv_test.shape}")


Training data shape: (40908, 95096)
Testing data shape: (13637, 95096)


In [17]:
from sklearn.linear_model import LogisticRegression

# Initialize Logistic Regression model
LR = LogisticRegression()

# Train the model on TF-IDF transformed training data
LR.fit(xv_train, y_train)

# Print model training status
print("Logistic Regression model trained successfully!")


Logistic Regression model trained successfully!


In [18]:
from sklearn.metrics import classification_report

# Predict labels for test data
pred_lr = LR.predict(xv_test)

# Print model accuracy
print("Model Accuracy:", LR.score(xv_test, y_test))

# Print detailed classification report
print(classification_report(y_test, pred_lr))


Model Accuracy: 0.9768277480384249
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      7045
           1       0.99      0.97      0.98      6592

    accuracy                           0.98     13637
   macro avg       0.98      0.98      0.98     13637
weighted avg       0.98      0.98      0.98     13637



In [19]:
from sklearn.tree import DecisionTreeClassifier

# Initialize the Decision Tree model
DT = DecisionTreeClassifier()

# Train the model on the training data
DT.fit(xv_train, y_train)


DecisionTreeClassifier()

In [20]:
from sklearn.metrics import classification_report

# Predict using the trained Decision Tree model
pred_dt = DT.predict(xv_test)

# Calculate the accuracy score
dt_score = DT.score(xv_test, y_test)

# Print model accuracy
print("Model Accuracy:", DT.score(xv_test,y_test))

# Print classification report
print(classification_report(y_test, pred_dt))


Model Accuracy: 0.9857006673021925
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7045
           1       1.00      0.97      0.99      6592

    accuracy                           0.99     13637
   macro avg       0.99      0.99      0.99     13637
weighted avg       0.99      0.99      0.99     13637



In [21]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report


In [22]:
xgb = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
xgb.fit(xv_train, y_train)


c:\Users\Moksha's Lappy\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [11:34:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [23]:
pred_xgb = xgb.predict(xv_test)


In [ ]:
print("XGBoost Model Performance:")
# Print model accuracy
print("Model Accuracy:", xgb.score(xv_test,y_test))
print(classification_report(y_test, pred_xgb))


XGBoost Model Performance:
Model Accuracy: 0.9869472757937963
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7045
           1       1.00      0.98      0.99      6592

    accuracy                           0.99     13637
   macro avg       0.99      0.99      0.99     13637
weighted avg       0.99      0.99      0.99     13637



In [9]:
import numpy as np

def output_label(text):
    """
    Function to predict whether the news is Fake or True.
    Returns: "Fake News" or "True News"
    """
    prediction = model.predict(vectorizer.transform([text]))  # Predict using the trained model
    return "Fake News" if prediction == 0 else "True News"

def generate_confidence_score(news_text, xgb, DT, LR, xv_test, y_test):
    """
    Generates a confidence score for a given news article.
    
    Parameters:
        news_text (str): The article text to evaluate.
        xgb: Trained XGBoost model.
        DT: Trained Decision Tree model.
        LR: Trained Logistic Regression model.
        xv_test: Feature vector for testing.
        y_test: True labels for testing.

    Returns:
        float: Confidence score (0-100)
    """
    # Check if the news is predicted as Fake
    if output_label(news_text) == "Fake News":
        return 0  # Confidence is 0 for Fake News

    # Compute average confidence score from all models
    confidence_score = (xgb.score(xv_test, y_test) + DT.score(xv_test, y_test) + LR.score(xv_test, y_test)) / 3
    confidence_score *= 100  # Convert to percentage

    return confidence_score

# Example Usage:
news_text = "Breaking: Government announces new policy to boost economy."
confidence = generate_confidence_score(news_text, xgb, DT, LR, xv_test, y_test)
print("Confidence Score:", confidence)



NameError: name 'xgb' is not defined

In [8]:
news = str(input("Enter the news article: "))
manual_testing(news)


NameError: name 'wordopt' is not defined